In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
import os

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

Need to dowaload the baci trade data from https://www.cepii.fr/CEPII/en/bdd_modele/bdd_modele_item.asp?id=37

this was under the HS17 codes

In [5]:
# bring in the data

country_codes = pd.read_csv("./BACI-data/country_codes_V202501.csv", dtype = {"country_code":str})

trade_data = pd.read_csv("./BACI-data/BACI_HS17_Y2017_V202501.csv")

trade_data = trade_data.rename(columns={
    't': 'year',
    'i': 'exporter',
    'j': 'importer',
    'k': 'product',
    'v': 'value',
    'q': 'quantity'
})

cntry_names = ['AUS','AUT','BEL','BRA','CAN','CHN','COL','DNK','FIN','FRA','DEU'
            ,'GRC','IND','IDN','IRL','ITA','JPN','KOR','MEX','NOR','POL','PRT'
            ,'RUS','ESP','SWE','CHE','THA','TUR','GBR','USA']

In [6]:
country_codes = pd.read_csv("./BACI-data/country_codes_V202501.csv", 
                            dtype = {"country_code":int})

country_codes.drop(columns=['country_iso2'], inplace=True)

country_codes.head()

,country_code,country_name,country_iso3
0,4,Afghanistan,AFG
1,8,Albania,ALB
2,12,Algeria,DZA
3,16,American Samoa,ASM
4,20,Andorra,AND


In [7]:
trade_data.head()

trade_data["value"] = trade_data["value"]*1000

In [8]:
# only grab manufactuing products

trade_data['hs2'] = trade_data['product'].astype(str).str[:2].astype(int)

trade_data = trade_data[(trade_data['hs2'] > 27) & (trade_data['hs2'] <= 96)]


In [9]:
#group things by year, exporter, and importer and add up the values

trade_data_collapsed = trade_data.groupby(['year', 
                                           'exporter', 
                                           'importer'], as_index=False)['value'].sum()

In [10]:
# now lets make this usefull for 

trade_data_collapsed = trade_data_collapsed.merge(
    country_codes.rename(columns={'country_code': 'exporter'}), on='exporter', how='left')

trade_data_collapsed = trade_data_collapsed.rename(columns={
    'country_name': 'exporter_name',
    'country_iso3': 'exporter_iso3'
})

In [11]:
trade_data_collapsed = trade_data_collapsed.merge(
    country_codes.rename(columns={'country_code': 'importer'}), on='importer', how='left')

trade_data_collapsed = trade_data_collapsed.rename(columns={
    'country_name': 'importer_name',
    'country_iso3': 'importer_iso3'
})

In [12]:
trade_data_just30 = trade_data_collapsed[trade_data_collapsed['exporter_iso3'].isin(cntry_names) &
                                         trade_data_collapsed['importer_iso3'].isin(cntry_names)].copy(deep = True)

In [13]:
trade_data_just30.drop(columns=['exporter', 'importer', "year", "exporter_name", "importer_name"], inplace=True)

trade_data_just30.rename(columns={'exporter_iso3': 'exporter',
                                                    'importer_iso3': 'importer'}, inplace=True)

trade_data_just30.reset_index(drop=True, inplace=True)

trade_data_just30.head()

,value,exporter,importer
0,4.182295e+07,AUS,AUT
1,2.260788e+08,AUS,BEL
2,1.990274e+08,AUS,BRA
3,1.047132e+09,AUS,CAN
4,1.995008e+10,AUS,CHN


In [14]:
trade_data_just30.to_csv("./BACI-data/top_30_trade_2017.csv", index = False)